# 🧪 Step 1: Research & Data Modelling
**PR Branch Name:** banks-data-modelling

This notebook documents the process for Step 1 of the "Banks in Berlin" project:
- **1.1 Data Source Discovery**
- **1.2 Modelling & Planning**
- **1.3 Prepare the /sources Directory**
- **1.4 Review**

Goal:
- Identify and document relevant data sources.
- Select the 23 key parameters for our use case.
- Draft the planned table schema.
- Plan cleaning and transformation steps before database population.


## 1.1 Data Source Discovery

**Topic:** Banks in Berlin

**Main source:**
- **Name:** OpenStreetMap (OSM) via OSMnx library
- **Source and origin:** Public crowdsourced geospatial database
- **Update frequency:** Continuous (dynamic)
- **Data type:** Dynamic (API query using `amenity=bank`)
- **Reason for selection:**  
  - Covers all banks in Berlin  
  - Includes coordinates, names, addresses, and other useful attributes  
  - Open, free, and easy to query programmatically

**Optional additional sources:**
- **Name:** Berlin Open Data Portal (daten.berlin.de)
- **Source and origin:** Official Berlin city government
- **Update frequency:** Varies per dataset
- **Data type:** Static or semi-static (download as CSV/GeoJSON)
- **Possible usage:** Enrich with official administrative boundaries or extra metadata

**Enrichment potential:**
- Neighborhood/district info from Berlin shapefiles (GeoJSON)
- Linking to local amenities for spatial context


In [2]:
# Install Libraries

 %pip install osmnx geopandas pandas --quiet

IndentationError: unexpected indent (1514150994.py, line 3)

In [4]:
# Import Libraries

import osmnx as ox
import geopandas as gpd
import pandas as pd

In [5]:
# Fetch banks in Berlin from OSM using the tag "amenity=bank"
# tags filter for only features with 

tags = {"amenity": "bank"}

In [6]:
# Fetch geometries for Berlin
# bank-gdf = GeoDataFrame (DataFrame with geometry)

banks_gdf = ox.features_from_place("Berlin, Germany", tags)


In [7]:
# Display basic info

print(f"Number of bank entries fetched: {len(banks_gdf)}")
banks_gdf.head(3)

Number of bank entries fetched: 323


geometry addr:city addr:country  \
element id                                                       
node    28968292   POINT (13.32 52.487)    Berlin           DE   
        60848455   POINT (13.471 52.53)    Berlin          NaN   
        87040399  POINT (13.389 52.511)       NaN          NaN   

                 addr:housenumber addr:postcode          addr:street  \
element id                                                             
node    28968292               42         10713      Berliner Straße   
        60848455               13         10369  Anton-Saefkow-Platz   
        87040399              NaN           NaN                  NaN   

                  addr:suburb amenity  atm branch  ... operator:type  \
element id                                         ...                 
node    28968292  Wilmersdorf    bank  yes     eG  ...           NaN   
        60848455          NaN    bank  yes    NaN  ...           NaN   
        87040399          NaN    bank  NaN    NaN  ...           NaN   

                 start_date building:levels roof:levels roof:shape indoor  \
element id                                                                  
node    28968292        NaN             NaN         NaN        NaN    NaN   
        60848455        NaN             NaN         NaN        NaN    NaN   
        87040399        NaN             NaN         NaN        NaN    NaN   

                 access room western_union building:part  
element id                                                
node    28968292    NaN  NaN           NaN           NaN  
        60848455    NaN  NaN           NaN           NaN  
        87040399    NaN  NaN           NaN           NaN  

[3 rows x 100 columns]

In [8]:
banks_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 323 entries, ('node', 28968292) to ('way', 611744021)
Data columns (total 100 columns):
 #   Column                                 Non-Null Count  Dtype   
---  ------                                 --------------  -----   
 0   geometry                               323 non-null    geometry
 1   addr:city                              227 non-null    object  
 2   addr:country                           141 non-null    object  
 3   addr:housenumber                       224 non-null    object  
 4   addr:postcode                          228 non-null    object  
 5   addr:street                            233 non-null    object  
 6   addr:suburb                            136 non-null    object  
 7   amenity                                323 non-null    object  
 8   atm                                    251 non-null    object  
 9   branch                                 9 non-null      object  
 10  brand                

## 1.2 Modelling & Planning

### Selected 23 Key Columns
1. osm_id
2. name
3. brand
4. operator
5. street
6. housenumber
7. postcode
8. city
9. country
10. phone
11. email
12. website
13. opening_hours
14. atm
15. wheelchair
16. building
17. latitude
18. longitude
19. geom_type
20. geom
21. neighbourhood
22. district
23. source

---

### How this connects to existing tables:
- **Coordinates (latitude, longitude, geom):** link to neighbourhood and district polygons.
- **Neighbourhood & district fields:** join with administrative boundaries table.
- **Source field:** ensures traceability.

---

### Planned Schema: `banks_in_berlin`
| Column Name     | Data Type | Description | Example |
|-----------------|-----------|-------------|---------|
| osm_id          | int       | Unique OSM ID | 12345678 |
| name            | text      | Bank name | Deutsche Bank |
| brand           | text      | Brand name if available | Sparkasse |
| operator        | text      | Entity operating the bank | Berliner Volksbank |
| street          | text      | Street name | Friedrichstraße |
| housenumber     | text      | House number | 45 |
| postcode        | text      | Postal code | 10117 |
| city            | text      | City name | Berlin |
| country         | text      | Country code | DE |
| phone           | text      | Contact phone | +49 30 123456 |
| email           | text      | Contact email | info@bank.de |
| website         | text      | Website URL | www.bank.de |
| opening_hours   | text      | Opening hours string | Mo-Fr 09:00-17:00 |
| atm             | text      | Presence of ATM | yes |
| wheelchair      | text      | Accessibility info | yes |
| building        | text      | Building type | yes |
| latitude        | float     | Latitude coordinate | 52.5200 |
| longitude       | float     | Longitude coordinate | 13.4050 |
| geom_type       | text      | Geometry type | Point |
| geom            | geometry  | Full geometry | (GeoJSON) |
| neighbourhood   | text      | Local neighbourhood name | Mitte |
| district        | text      | Berlin district | Mitte |
| source          | text      | Data source info | OSM |

---

### Known Data Issues
- Missing contact details for some entries.
- Inconsistent postcode and address formats.
- Neighbourhood and district not always included in raw OSM data.
- Opening hours in non-standard formats.

---

### Transformation Plan
1. Fetch data from OSM with filter `amenity=bank` (Berlin bounding box).
2. Clean column names → snake_case.
3. Normalize formats (phone, postcode, website URLs).
4. Enrich with neighbourhood/district via spatial join.
5. Save cleaned dataset (GeoJSON + CSV).


In [7]:
# Select 23 Columns & Add Coordinates

In [9]:
# Ensure geometry type is Point for lat/lon extraction

banks_gdf = banks_gdf.to_crs(epsg=4326)

In [18]:
banks_gdf['geometry'] = banks_gdf['geometry'].apply(lambda geom: geom if geom.geom_type == 'Point' else geom.representative_point())
#Extract latitude and longitude
banks_gdf["latitude"] = banks_gdf.geometry.y
banks_gdf["longitude"] = banks_gdf.geometry.x
banks_gdf

geometry addr:city addr:country  \
element id                                                        
node    28968292    POINT (13.32 52.487)    Berlin           DE   
        60848455    POINT (13.471 52.53)    Berlin          NaN   
        87040399   POINT (13.389 52.511)       NaN          NaN   
        89274635   POINT (13.416 52.523)    Berlin          NaN   
        203561614  POINT (13.538 52.528)    Berlin          NaN   
...                                  ...       ...          ...   
way     210920895  POINT (13.491 52.421)    Berlin           DE   
        336063499   POINT (13.434 52.51)    Berlin           DE   
        422475544  POINT (13.385 52.531)    Berlin           DE   
        423847739  POINT (13.199 52.534)       NaN          NaN   
        611744021  POINT (13.316 52.513)    Berlin           DE   

                  addr:housenumber addr:postcode          addr:street  \
element id                                                              
node    28968292                42         10713      Berliner Straße   
        60848455                13         10369  Anton-Saefkow-Platz   
        87040399               NaN           NaN                  NaN   
        89274635                 5         10178      Alexanderstraße   
        203561614              1/2         12681  Helene-Weigel-Platz   
...                            ...           ...                  ...   
way     210920895               15         12357            Alt-Rudow   
        336063499                5         10243        Am Ostbahnhof   
        422475544               28         10115      Invalidenstraße   
        423847739              NaN           NaN                  NaN   
        611744021               10         10625        Leibnizstraße   

                      addr:suburb amenity  atm branch  ... building:levels  \
element id                                             ...                   
node    28968292      Wilmersdorf    bank  yes     eG  ...             NaN   
        60848455              NaN    bank  yes    NaN  ...             NaN   
        87040399              NaN    bank  NaN    NaN  ...             NaN   
        89274635              NaN    bank  yes    NaN  ...             NaN   
        203561614             NaN    bank  yes    NaN  ...             NaN   
...                           ...     ...  ...    ...  ...             ...   
way     210920895           Rudow    bank  yes    NaN  ...             NaN   
        336063499  Friedrichshain    bank  yes    NaN  ...             NaN   
        422475544           Mitte    bank  NaN    NaN  ...             NaN   
        423847739             NaN    bank  yes    NaN  ...             NaN   
        611744021  Charlottenburg    bank  NaN    NaN  ...             NaN   

                  roof:levels roof:shape indoor     access  room  \
element id                                                         
node    28968292          NaN        NaN    NaN        NaN   NaN   
        60848455          NaN        NaN    NaN        NaN   NaN   
        87040399          NaN        NaN    NaN        NaN   NaN   
        89274635          NaN        NaN    NaN        NaN   NaN   
        203561614         NaN        NaN    NaN        NaN   NaN   
...                       ...        ...    ...        ...   ...   
way     210920895         NaN        NaN    NaN        NaN   NaN   
        336063499         NaN        NaN   room  customers  shop   
        422475544         NaN        NaN    NaN        NaN   NaN   
        423847739         NaN        NaN   room        NaN   NaN   
        611744021         NaN        NaN    NaN        NaN   NaN   

                  western_union building:part   latitude  longitude  
element id                                                           
node    28968292            NaN           NaN  52.486668  13.319723  
        60848455            NaN           NaN  52.530331  13.471037  
        87040399            Na

In [30]:
# Select the 23 columns (fill missing with None if not present)

selected_columns = [
    #"osmid",
    "name", "brand", "operator",
    "addr:street", "addr:housenumber", "addr:postcode", "addr:city", "addr:country",
    "phone", "email", "website", "opening_hours",
    "atm", "wheelchair", "building",
    "latitude", "longitude", "geometry",
    # placeholders for enrichment
    #"neighbourhood", "district",
    # add source info
    "source"
]

In [25]:
# Rename map for only the columns that need renaming

rename_map = {
    "addr:street": "street",
    "addr:housenumber": "housenumber",
    "addr:postcode": "postcode",
    "addr:city": "city",
    "addr:country": "country"
}

In [ ]:
# # Add missing columns if they don’t exist in the data
# for col in selected_columns:
#     if col not in banks_gdf.columns:
#         banks_gdf[col] = None

In [31]:
# Select the columns in the right order
banks_df = banks_gdf[selected_columns]

In [32]:
# Rename the columns
banks_df = banks_df.rename(columns=rename_map)

In [33]:
# Preview the final DataFrame
banks_df.head()

name               brand            operator  \
element id                                                                      
node    28968292   Berliner Volksbank  Berliner Volksbank                 NaN   
        60848455            Sparkasse                 NaN  Berliner Sparkasse   
        87040399                  DKB                 NaN                 NaN   
        89274635        Deutsche Bank       Deutsche Bank       Deutsche Bank   
        203561614           Sparkasse                 NaN  Berliner Sparkasse   

                                street housenumber postcode    city country  \
element id                                                                    
node    28968292       Berliner Straße          42    10713  Berlin      DE   
        60848455   Anton-Saefkow-Platz          13    10369  Berlin     NaN   
        87040399                   NaN         NaN      NaN     NaN     NaN   
        89274635       Alexanderstraße           5    10178  Berlin     NaN   
        203561614  Helene-Weigel-Platz         1/2    12681  Berlin     NaN   

                  phone email website  \
element id                              
node    28968292    NaN   NaN     NaN   
        60848455    NaN   NaN     NaN   
        87040399    NaN   NaN     NaN   
        89274635    NaN   NaN     NaN   
        203561614   NaN   NaN     NaN   

                                                       opening_hours  atm  \
element id                                                                  
node    28968292   Mo-Fr 10:00-13:00, Mo 14:00-16:00, Tu,Th 14:00...  yes   
        60848455             Mo,We,Fr 09:30-15:00; Tu,Th 09:30-18:00  yes   
        87040399                                                 NaN  NaN   
        89274635   Mo-Tu 10:00-18:00; We 10:00-16:00; Th 10:00-18...  yes   
        203561614            Mo,We,Fr 09:30-15:00; Tu,Th 09:30-18:00  yes   

                  wheelchair building   latitude  longitude  \
element id                                                    
node    28968292         yes      NaN  52.486668  13.319723   
        60848455     limited      NaN  52.530331  13.471037   
        87040399     limited      NaN  52.511050  13.388798   
        89274635         yes      NaN  52.523238  13.415750   
        203561614        yes      NaN  52.527687  13.538327   

                                geometry source  
element id                                       
node    28968292    POINT (13.32 52.487)    NaN  
        60848455    POINT (13.471 52.53)    NaN  
        87040399   POINT (13.389 52.511)    NaN  
        89274635   POINT (13.416 52.523)    NaN  
        203561614  POINT (13.538 52.528)    NaN

In [34]:
# Info about the final DataFrame
banks_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
MultiIndex: 323 entries, ('node', 28968292) to ('way', 611744021)
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   name           322 non-null    object  
 1   brand          194 non-null    object  
 2   operator       174 non-null    object  
 3   street         233 non-null    object  
 4   housenumber    224 non-null    object  
 5   postcode       228 non-null    object  
 6   city           227 non-null    object  
 7   country        141 non-null    object  
 8   phone          28 non-null     object  
 9   email          2 non-null      object  
 10  website        46 non-null     object  
 11  opening_hours  278 non-null    object  
 12  atm            251 non-null    object  
 13  wheelchair     287 non-null    object  
 14  building       8 non-null      object  
 15  latitude       323 non-null    float64 
 16  longitude      323 non-null    float64 


In [17]:
# How many rows and columns?
# banks_df.shape

print("Rows, Columns:", banks_df.shape)

Rows, Columns: (323, 22)


In [18]:
# What are the column names (in order)?
# banks_df.columns.tolist()

print("\nColumns:", banks_df.columns.tolist())


Columns: ['osm_id', 'name', 'brand', 'operator', 'street', 'housenumber', 'postcode', 'city', 'country', 'phone', 'email', 'website', 'opening_hours', 'atm', 'wheelchair', 'building', 'latitude', 'longitude', 'geometry', 'neighbourhood', 'district', 'source']


In [19]:
# What data types does pandas/GeoPandas see?
# banks_df.dtypes

print("\nDtypes:\n", banks_df.dtypes)


Dtypes:
 osm_id             object
name               object
brand              object
operator           object
street             object
housenumber        object
postcode           object
city               object
country            object
phone              object
email              object
website            object
opening_hours      object
atm                object
wheelchair         object
building           object
latitude           object
longitude          object
geometry         geometry
neighbourhood      object
district           object
source             object
dtype: object


In [20]:
# Count the rows

row_count = len(banks_df)
print(row_count)

323


In [21]:
# Count missing values (NaN/None) in each column

missing_count = banks_df.isna().sum().sort_values(ascending=False)
print()


## 1.3 Prepare the /sources Directory

- **Raw Data Files:**  
    - `banks_raw.geojson` (includes geometry)  
    - `banks_raw.csv` (tabular only, no geometry)  

- **README.md** in `/sources` will contain:
    - Data sources used.
    - Planned transformation steps.


In [22]:
# Save as GeoJSON (keeps geometry) and CSV

raw_geojson_path = "../sources/banks_raw.geojson"
raw_csv_path = "../sources/banks_raw.csv"


banks_gdf.to_file(raw_geojson_path, driver="GeoJSON")
banks_gdf.drop(columns="geometry").to_csv(raw_csv_path, index=False)

print(f"Raw data saved to: {raw_geojson_path} and {raw_csv_path}")

Raw data saved to: ../sources/banks_raw.geojson and ../sources/banks_raw.csv


## 1.4 Review

- All 23 target columns defined.
- Data sources identified and documented.
- Schema draft created.
- Data fetched and stored in `/sources`.
- Data cleaning & enrichment plan in place.

**Next Step:** Step 2 — Fetch & Transform data.
